In [ ]:
import tensorflow as tf
import numpy as np
from lhrNetUtils.data_loader import lhrNetDataLoader

In [ ]:
testDataLoader = lhrNetDataLoader("data/test_details.csv",8)
trainDataLoader = lhrNetDataLoader("data/train_details.csv",8)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,64)),
    tf.keras.layers.Dense(2048,activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256,activation="relu"),
    tf.keras.layers.Dense(2,activation="softmax")
])

loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer="adam",loss=loss_fn,metrics=["accuracy"])

In [ ]:
model.fit(trainDataLoader,epochs=15)

In [ ]:
result = model.evaluate(testDataLoader)
dict(zip(model.metrics_names, result))

In [ ]:
model.predict(np.array([np.genfromtxt("data/1731758527.csv" ,delimiter=",")]))